In [20]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
far.utils.check()

2


In [73]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., 0.]])
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape())

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

In [74]:


io_optim = far.MomentumOptimizer(far.get_hyperparameter('eta', 0.1), 
                                   far.get_hyperparameter('mu', .9))

# io_optim_dict = io_optim.minimize(cost) 

oo = tf.reduce_mean(v1*v2)
# rhg = far.ReverseHg().compute_gradients(io_optim_dict, oo)

# optim_oo = tf.train.AdamOptimizer(.01)
# ts_hy = optim_oo.apply_gradients(rhg.hgrad_hvar)

In [75]:
far.HyperOptimizer().set_dynamics(io_optim, cost)

In [76]:
hyper_optim = tf.train.AdamOptimizer()
farho = far.HyperOptimizer().set_dynamics(io_optim, cost).set_problem(
    hyper_optim, oo)

[<tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>, <tf.Variable 'eta:0' shape=() dtype=float32_ref>, <tf.Variable 'mu:0' shape=() dtype=float32_ref>]

defaultdict(<function ReverseHg.__init__.<locals>.<lambda> at 0x7fad75946d90>, {<tf.Variable 'mu:0' shape=() dtype=float32_ref>: [<tf.Variable 'mu/hypergradient:0' shape=() dtype=float32_ref>], <tf.Variable 'eta:0' shape=() dtype=float32_ref>: [<tf.Variable 'eta/hypergradient:0' shape=() dtype=float32_ref>], <tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>: [<tf.Variable 'lambda/hypergradient:0' shape=(2, 2) dtype=float32_ref>]})

[True, True, True]


In [77]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.1, 0.89999998]

In [78]:
farho.run(10, _skip_hyper_ts=True)

ss.run(far.utils.hypergradients() + [oo])


[array([[ 0.22334358,  1.34006155],
        [-0.22334358,  0.        ]], dtype=float32),
 13.395197,
 0.28557456,
 0.0038858843]

In [79]:
# ss.run(ts_hy)  # perform an hypergradient descent step....
# print(ss.run(far.utils.hyperparameters()))
# print(ss.run(oo))

# check with rfho

In [80]:
import rfho as rf

Experiment save directory is  /media/luca/DATA/EXPERIMENTS
Data folder is /media/luca/DATA/DATASETS


In [81]:
w, c, co = rf.vectorize_model([v1, v2], cost, oo, augment=1)

In [82]:
dyn = rf.MomentumOptimizer.create(w, 0.1, 0.9, loss=c)

In [83]:
hyperg = rf.HyperOptimizer(dyn, {co: lmbd}, rf.ReverseHG)

In [84]:
hyperg.initialize()

True

In [85]:
hgs = hyperg.run(10, val_feed_dict_suppliers={co:lambda: None},
                 _debug_no_hyper_update=True)

In [86]:
ss.run(hyperg.hyper_gradients.hyper_gradient_vars)

[array([[ 0.2233436 ,  1.34006178],
        [-0.2233436 ,  0.        ]], dtype=float32)]

# END... RESULTS ARE (almost the) SAME (CFR cell 306)

In [209]:
rfho_hs = hyperg.hyper_gradients.w_hist

In [210]:
len(rfho_hs)

10

In [218]:
rfho_hs[9], rhg._history[10]  # in far version also last value is saved

(array([-24.56974983, -22.56974983, -21.56974983,   0.27419662,
          0.54839325,  -0.27419662,   0.        ,  44.92248917,
         44.92248917,  44.92248917,   2.48619032,   4.97238111,
         -2.48619032,   0.        ], dtype=float32),
 [array([-29.34610748, -27.34610748, -26.34610748], dtype=float32),
  array([ 47.76357651,  47.76357651,  47.76357651], dtype=float32),
  array([[-0.00439989, -0.00879972],
         [ 0.00439989,  0.        ]], dtype=float32),
  array([[ 2.78596425,  5.57192898],
         [-2.78596425,  0.        ]], dtype=float32)])

In [219]:
[1,2,4][:-1]

[1, 2]

In [222]:
ss.run(list(hyperg.hyper_gradients.p_dict.values()))

[array([  7.33333349e+00,   7.33333349e+00,   7.33333349e+00,
         -3.87176406e-05,  -7.74352811e-05,   3.87176406e-05,
          0.00000000e+00,  -4.29872227e+00,  -4.29872227e+00,
         -4.29872227e+00,  -1.10320083e-03,  -2.20641680e-03,
          1.10320083e-03,   0.00000000e+00], dtype=float32)]